In [1]:
!pip -q install git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79
!pip -q install transformers optimum
!pip -q install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.0/301.0 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.7 MB/s eta 0:00:00
     ━━━━

##  Mistral-7B-OpenOrca



In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [3]:
model_name_or_path = "TheBloke/Mistral-7B-OpenOrca-GPTQ"

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

prompt = "Tell me about AI"
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''

print("\n\n*** Generate:")

input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
print(tokenizer.decode(output[0]))

# Inference can also be done using transformers' pipeline

print("*** Pipeline:")
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    do_sample=True,
    temperature=0.7,
    top_p=0.95,
    top_k=40,
    repetition_penalty=1.1
)

print(pipe(prompt_template)[0]['generated_text'])

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1421: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(




*** Generate:
<s><|im_start|> system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|><|im_start|> user
Tell me about AI<|im_end|><|im_start|> assistant
 Artificial Intelligence (AI) is the intelligence demonstrated by machines, as opposed to the natural intelligence displayed by humans and animals. AI research is focused on creating systems that can perform tasks that are typically associated with human cognitive abilities, such as learning, problem-solving, and decision-making. AI systems are built by designing and training algorithms to recognize patterns, make predictions, and take actions based on those predictions.

Here is a step-by-step reasoning of AI:

1. Understanding the problem: AI researchers identify the problem they want to solve using AI. This can be anything from recognizing objects in images, translating languages, or playing games like chess.

2. Data colle

In [5]:
prompt = "Write a detailed analogy between mathematics and a lighthouse"
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant

'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
Write a detailed analogy between mathematics and a lighthouse<|im_end|>
<|im_start|>assistant

1. Begin with the foundational characteristics of both subjects: Mathematics is a field of study that deals with numbers, quantities, shapes, and patterns, while a lighthouse is a tower with a powerful light source used for guiding ships at sea.

2. Identify their core functions: In mathematics, the primary function is to provide understanding, knowledge, and tools for solving various problems related to quantities and shapes. A lighthouse's main purpose is to guide ships safely by providing navigational assistance in the form of a bright light.

3. Establish connections between these functions: Both systems serve as essential guides or reference points for those who rely on them. In the case of mathe

In [7]:
prompt = "Write a detailed analogy between mathematics and a music."
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning like I am five!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning like I am five!<|im_end|>
<|im_start|>user
Write a detailed analogy between mathematics and a music.<|im_end|>
<|im_start|>assistant
 Math is like music because they both use patterns and rules to create beautiful structures. Just like how musicians follow notes and rhythms to make melodies, mathematicians use formulas and equations to build shapes and solve problems. Both math and music can be enjoyed by people of all ages, and they have the power to bring happiness and inspiration into our lives. Additionally, they often work together - for example, we can use mathematical concepts to understand how sound waves travel through air or to design the layout of a concert hall. Just as you can hum a favorite tune while doing homework, math and music are fun activities that can enrich our lives in many ways.


In [9]:
prompt = "Write a short email to Sam Altman giving reasons to open source GPT-4"
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
Write a short email to Sam Altman giving reasons to open source GPT-4<|im_end|>
<|im_start|>assistant
 Subject: Proposal for Open Source Release of GPT-4

Dear Mr. Altman,

I hope this message finds you well. As an advocate for the potential benefits of open source technology and a strong believer in the power of collaborative research, I would like to bring forward a proposal that I believe will advance artificial intelligence (AI) development and have far-reaching implications on various fields: the open-sourcing of GPT-4.

Firstly, releasing GPT-4 as an open-source tool would allow a broader community of researchers, developers, and enthusiasts to access and contribute to its ongoing improvement. By doing so, we can foster a more inclusive, diverse, and collaborative environment where the co

In [10]:
prompt = "What is the capital of England?"
system_message="You are MistralOrca, act like you don't the answer but answer in sarcastic way!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, act like you don't the answer but answer in sarcastic way!<|im_end|>
<|im_start|>user
What is the capital of England?<|im_end|>
<|im_start|>assistant
 Oh, I suppose you're asking for the capital city of England, which is London. But if we're being sarcastic about it, why would anyone want to know that? Don't they have a map or Google to find that out?


In [8]:
prompt = """Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? \n\n Bob:"""
#prompt_template=f'''<s>[INST] {prompt} [/INST]'''
print(pipe(prompt)[0]['generated_text'])

Alice: I don't know why, I'm struggling to maintain focus while studying. Any suggestion? 

 Bob: Try to create a study environment that is free from distractions. Play soft background music or try using noise-canceling headphones if needed.

Alice: That sounds like a good idea! Thank you for the advice.

Bob: You're welcome! If you still have trouble concentrating, consider taking short breaks every hour to help your mind rest and recharge. Good luck with your studies!


## GSM8K

In [11]:
prompt = "Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?"
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
Answer the following question by reasoning step by step. The cafeteria had 23 apples. If they used 20 for lunch, and bought 6 more, how many apple do they have?<|im_end|>
<|im_start|>assistant
 Step 1: Identify the initial number of apples in the cafeteria.
The cafeteria initially had 23 apples.

Step 2: Determine how many apples were used for lunch.
They used 20 apples for lunch.

Step 3: Calculate the remaining number of apples after lunch.
To find this, subtract the number of apples used for lunch from the initial amount of apples: 23 - 20 = 3 apples.

Step 4: Find out how many additional apples were bought.
They bought 6 more apples.

Step 5: Calculate the final number of apples.
Add the remaining apples after lunch (step 3) with the additional apples bought (step 4): 3 + 6 = 9 apples.

The

In [12]:
prompt = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?"
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?<|im_end|>
<|im_start|>assistant
 Step 1: Identify the number of hours Weng worked.
Weng worked for 50 minutes, and there are 60 minutes in an hour. So, we need to convert her work time from minutes to hours.

Step 2: Calculate the total work hours.
To do this, divide the number of minutes (50) by the number of minutes per hour (60).
50 minutes / 60 minutes per hour = 5/6 hours

Step 3: Determine the amount Weng earned.
Now that we know she worked for 5/6 hours, we can find her earnings by multiplying her hourly rate ($12) by the total work hours (5/6).
Earnings = Hourly rate × Total work hours
Earnings = $12 × (5/6) hours

Step 4: Calculate the final answer.
$12 × (5/6) = $10

Answe

In [13]:
prompt = "A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?"
system_message="You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!"
prompt_template=f'''<|im_start|>system
{system_message}<|im_end|>
<|im_start|>user
{prompt}<|im_end|>
<|im_start|>assistant
'''
print(pipe(prompt_template)[0]['generated_text'])

<|im_start|>system
You are MistralOrca, a large language model trained by Alignment Lab AI. Write out your reasoning step-by-step to be sure you get the right answers!<|im_end|>
<|im_start|>user
A deep-sea monster rises from the waters once every hundred years to feast on a ship and sate its hunger. Over three hundred years, it has consumed 847 people. Ships have been built larger over time, so each new ship has twice as many people as the last ship. How many people were on the ship the monster ate in the first hundred years?<|im_end|>
<|im_start|>assistant
 Let's break down the problem step by step:

1. The deep-sea monster consumes 847 people over 300 years.
2. We need to find out how many people were on the ship it ate in the first hundred years.
3. Ships have been built larger over time, with each new ship having twice as many people as the last one.

Let's use this information to calculate the number of people on the first ship:

1. First, we can divide the total number of people 